In [1]:
import os
import pandas as pd

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show, save
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.palettes import Spectral11, colorblind, Inferno, BuGn, brewer
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource,LinearColorMapper,BasicTicker, PrintfTickFormatter, ColorBar
import datetime
import os

In [3]:
folders = [f.name for f in os.scandir('.') if f.is_dir()][1:]

In [4]:
print(folders)

['001', '003', '004', '005', '006', '007', '008', '009', '010', '011', '013', '015', '016', '017', '019', '021', '024', '025', '026', '027', '028', '029', '030', '031', '032', '034', '037', '038', '039', '040', '041', '042', '044', '045', '047', '048', '049', '050', '052', '053', '055', '058', '059', '063', '065', '066', '068', '101', '102', '104', '105', '106', '107', '108', '109', '110', '112', '113', '114', '115', '116', '117', '121', '122', '123', '124', '125', '127', '128', '129', '130', '201', '202', '203', '204', '205', '206', '207', '208', '301_OFF', '301_ON', '302_OFF', '302_ON', '303', '304_OFF', '304_ON', '305', '401', '402', '403', '404', '405', '406', '407_OFF', '407_ON', '408', '409', '410', '411', '412', '413', '414', '415_OFF', '415_ON', '416', '417', '418', '419', '420_OFF', '420_ON', '421']


In [5]:
top_home_page = """
<!DOCTYPE html>
<html>
	<head>
		<title>Home Survey</title>
		<style type="text/css">
			ul{
				float:left;
				list-style: none;
			}
			body{
				font-size: 20px;
				font-family: Open Sans;
				line-height: 32px;
				text-align: center;
			}
			a{
				text-decoration: none;
				display: block;
				padding: 10px 38px 10px 38px;
				border: 1px solid grey;
				background-color: lightgrey;
			}
			a:hover{
				background-color: navy;
				color: lightgrey;
			}
		</style>
	</head>
	<body>
    <ul>
        <li class="fir"><strong>Florida</strong></li>
"""
bottom_home_page = """
	</body>
</html>
"""

In [6]:
#Create a list of houses
def createList(folders,borneInf,borneSup):
    liste = []
    for folder in folders :
        if (int(folder[0:3])>= borneInf) and (int(folder[0:3])<borneSup):
            liste +=[folder]
    return liste       

In [7]:
sw0 = createList(folders,0,100)
sw1 = createList(folders,100,200)
sw2 = createList(folders,200,300)
sw3 = createList(folders,300,400)
sw4 = createList(folders,400,500)

for f in sw4:
    top_home_page += """<li><a href='"""+f+"""/index.html'>"""+f+"""</a></li>"""
top_home_page += """
</ul>
    <ul>
        <li class="fir"><strong>Colorado</strong></li>"""
for f in sw3:
    top_home_page += """<li><a href='"""+f+"""/index.html'>"""+f+"""</a></li>"""
top_home_page += """
</ul>
    <ul>
        <li class="fir"><strong>Portland</strong></li>"""
for f in sw2:
    top_home_page += """<li><a href='"""+f+"""/index.html'>"""+f+"""</a></li>"""
top_home_page += """
</ul>
    <ul>
        <li class="fir"><strong> South California (HENGH)</strong></li>"""
for f in sw1:
    top_home_page += """<li><a href='"""+f+"""/index.html'>"""+f+"""</a></li>"""
top_home_page += """
</ul>
    <ul>
        <li class="fir"><strong> California (HENGH)</strong></li>"""
for f in sw0:
    top_home_page += """<li><a href='"""+f+"""/index.html'>"""+f+"""</a></li>"""
top_home_page += bottom_home_page

In [8]:
#writes to html file
hp = open('index.html','w')
hp.write(top_home_page)
hp.close()

In [9]:
#Sort columns of the DataTable
def sortDataTable(data):
    data['Time'] = pd.to_datetime(data['Time'], format='%m/%d/%Y%H%M')
    time = data["Time"].values
    forms = []
    pms = []
    intemps = []
    outtemps = []
    pm25s = []
    pm10s = []
    co2s = []
    inrhs = []
    outrhs = []
    ibus = []
    anms = []
    mtrs = []
    doors = []
    title_array = []
    #iterate each column (column name) in dataset
    for x in data[1:]:
        if x[0:3] == "FRM":
            forms += [x]
        elif x == 'PMin' or x == 'PMout':
            pms += [x]
        elif x == 'OUT_T' or x == 'OUT2_T':
            outtemps += [x]
        elif x[0:2] == 'T_' or x[len(x)-2:] == '_T':
            intemps += [x]
        elif x == 'OUT_RH' or x == 'OUT2_RH':
            outrhs += [x]
        elif x[0:3] == 'RH_' or x[len(x)-3:] == '_RH':
            inrhs += [x]
        elif x[len(x)-5:] == '_PM10':
            pm10s += [x]
        elif x[len(x)-5:] == '_PM25':
            pm25s += [x]
        elif x[0:4] == 'CO2_' or x[len(x)-4:] == '_CO2' :
            co2s += [x]
        elif x[0:4] == 'IBU_':
            ibus += [x]
        elif x[0:4] == 'ANM_':
            anms += [x]
        elif x[0:4] == 'MTR_':
            mtrs += [x]
        elif x[0:4] == 'STA_':
            doors += [x]

    return (time, forms,pms,intemps,outtemps,pm25s,pm10s,co2s,inrhs,outrhs,ibus,anms,mtrs,doors)

In [10]:
# accessing the CSV dataset
colarr = ['red', 'blue', 'black', 'orange', 'purple', 'grey', 'green', 'pink', 'yellow', 'maroon', 'brown', 'navy', 'magenta', 'tan']
for folder in folders[0:]:
    print('debut ' + folder)
    df = data = pd.read_csv(folder+"/DataTable"+folder+".csv", parse_dates=['Time'])
    time, forms,pms,intemps,outtemps,pm25s,pm10s,co2s,inrhs,outrhs,ibus,anms,mtrs,doors = sortDataTable(data)
    tophtml = """<!DOCTYPE html>
    <html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href="https://cdn.pydata.org/bokeh/release/bokeh-0.12.10.min.css" type="text/css" />
    <script type="text/javascript" src="https://cdn.pydata.org/bokeh/release/bokeh-0.12.10.min.js"></script>
    <script type="text/javascript">
        Bokeh.set_log_level("info");
    </script>
    <title>"""+folder+"""</title>
    <style>
    * {box-sizing: border-box}
    body {font-family: "Lato", sans-serif;}
    /* Style the tab */
    .tab {
      float: left;
      border: 1px solid #ccc;
      background-color: #f1f1f1;
      width: 30%;
      height: 300px;
    }
    /* Style the buttons inside the tab */
    .tab button {
      display: block;
      background-color: inherit;
      color: black;
      padding: 22px 16px;
      width: 100%;
      border: none;
      outline: none;
      text-align: left;
      cursor: pointer;
      transition: 0.3s;
      font-size: 17px;
    }
    /* Change background color of buttons on hover */
    .tab button:hover {
      background-color: #ddd;
    }
    th,td{
        padding: 10px 20px 10px 20px;
        border: 1px solid darkgrey;
    }
    /* Create an active/current "tab button" class */
    .tab button.active {
      background-color: #ccc;
    }
    /* Style the tab content */
    .tabcontent {
      float: left;
      padding: 0px 12px;
      /*border: 1px solid #ccc;*/
      width: 70%;
      border-left: none;
      height: 300px;
    }
    iframe{
      /*margin-left:200px;*/
      width:700px;
      height:620px;
    }
    </style>
    </head>
    <body>
    <h2></h2>
    <div class="tab">
      <button class="tablinks" onclick="openCity(event, 'pms')" id="defaultOpen">MetOne PM</button>"""
    navlist = ""
    graphs = """
    <div id="pms" class="tabcontent"><h3>MetOne PM</h3><iframe src="pm.html"></iframe>"""
    if pms:
        #add summary statistics
        #create bokeh plots
        output_file(folder+"/pm.html")
        plo = figure(title="MetOne PM", x_axis_label='Time', y_axis_label='ug/m3', x_axis_type='datetime')
        for t in range(0, len(pms)):
            plo.line(time, df[pms[t]].values, legend=pms[t], line_width=2, color =colarr[t])
            graphs+="""
            <p><b>"""+ pms[t] +"""<br></b>
            Mean: """+ str(round(df.loc[:,pms[t]].mean(),1))  + """<br>
            Minimum: """ + str(round(df.loc[:,pms[t]].min(),1)) + """<br>
            5th Percentile: """ + str(round(df.loc[:,pms[t]].quantile(q=0.05),1)) + """<br>
            25th Percentile: """ + str(round(df.loc[:,pms[t]].quantile(q=0.25),1)) + """<br>
            Median: """ + str(round(df.loc[:,pms[t]].quantile(q=0.50),1)) + """<br>
            75th Percentile: """ +str(round(df.loc[:,pms[t]].quantile(q=0.75),1))  + """<br>
            95th Percentile: """ + str(round(df.loc[:,pms[t]].quantile(q=0.95),1)) + """<br>
            Maximum: """ + str(round(df.loc[:,pms[t]].max(),1)) + """<br></p>
            """
        graphs+="</div>"
        save(plo)
    if pm25s:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'pm25s')">AirVisual Pro PM2.5</button>"""
        graphs += """
        <div id="pm25s" class="tabcontent"><h3>AirVisual Pro PM2.5</h3><iframe src="pm25.html"></iframe>"""
        #add summary statistics
        #create bokeh plots
        output_file(folder+"/pm25.html")
        plo = figure(title="AirVisual Pro PM2.5", x_axis_label='Time', y_axis_label='ug/m3', x_axis_type='datetime')
        for t in range(0, len(pm25s)):
            plo.line(time, df[pm25s[t]].values, legend=pm25s[t], line_width=2, color =colarr[t])
            graphs+="""
            <p><b>"""+ pm25s[t] +"""<br></b>
            Mean: """+ str(round(df.loc[:,pm25s[t]].mean(),1))  + """<br>
            Minimum: """ + str(round(df.loc[:,pm25s[t]].min(),1)) + """<br>
            5th Percentile: """ + str(round(df.loc[:,pm25s[t]].quantile(q=0.05),1)) + """<br>
            25th Percentile: """ + str(round(df.loc[:,pm25s[t]].quantile(q=0.25),1)) + """<br>
            Median: """ + str(round(df.loc[:,pm25s[t]].quantile(q=0.50),1)) + """<br>
            75th Percentile: """ +str(round(df.loc[:,pm25s[t]].quantile(q=0.75),1))  + """<br>
            95th Percentile: """ + str(round(df.loc[:,pm25s[t]].quantile(q=0.95),1)) + """<br>
            Maximum: """ + str(round(df.loc[:,pm25s[t]].max(),1)) + """<br></p>
            """
        graphs+="</div>"
        save(plo)
    if pm10s:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'pm10s')">AirVisual Pro PM10</button>"""
        graphs += """
        <div id="pm10s" class="tabcontent"><h3>AirVisual Pro PM10</h3><iframe src="pm10.html"></iframe>"""
        #add summary statistics
        #create bokeh plots
        output_file(folder+"/pm10.html")
        plo = figure(title="AirVisual Pro PM10", x_axis_label='Time', y_axis_label='ug/m3', x_axis_type='datetime')
        for t in range(0, len(pm10s)):
            plo.line(time, df[pm10s[t]].values, legend=pm10s[t], line_width=2, color =colarr[t])
            graphs+="""
            <p><b>"""+ pm10s[t] +"""<br></b>
            Mean: """+ str(round(df.loc[:,pm10s[t]].mean(),1))  + """<br>
            Minimum: """ + str(round(df.loc[:,pm10s[t]].min(),1)) + """<br>
            5th Percentile: """ + str(round(df.loc[:,pm10s[t]].quantile(q=0.05),1)) + """<br>
            25th Percentile: """ + str(round(df.loc[:,pm10s[t]].quantile(q=0.25),1)) + """<br>
            Median: """ + str(round(df.loc[:,pm10s[t]].quantile(q=0.50),1)) + """<br>
            75th Percentile: """ +str(round(df.loc[:,pm10s[t]].quantile(q=0.75),1))  + """<br>
            95th Percentile: """ + str(round(df.loc[:,pm10s[t]].quantile(q=0.95),1)) + """<br>
            Maximum: """ + str(round(df.loc[:,pm10s[t]].max(),1)) + """<br></p>
            """
        graphs+="</div>"
        save(plo)
        save(plo)
    #print('taille co2 av html:' + str(len(co2s)) + ' pour le fichier : ' + folder)
    if co2s:
        #print('est dans le if du html pour le fichier : ' + folder)
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'co2s')">CO2</button>"""
        graphs += """
        <div id="co2s" class="tabcontent"><h3>CO2</h3><iframe src="co2.html"></iframe>"""
        #add summary statistics
        #create bokeh plots
        output_file(folder+"/co2.html")
        plo = figure(title="CO2", x_axis_label='Time', y_axis_label='ppm', x_axis_type='datetime')
        for t in range(0, len(co2s)):
            plo.line(time, df[co2s[t]].values, legend=co2s[t], line_width=2, color =colarr[t])
            graphs+="""
            <p><b>"""+ co2s[t] +"""<br></b>
            Mean: """+ str(round(df.loc[:,co2s[t]].mean(),1))  + """<br>
            Minimum: """ + str(round(df.loc[:,co2s[t]].min(),1)) + """<br>
            5th Percentile: """ + str(round(df.loc[:,co2s[t]].quantile(q=0.05),1)) + """<br>
            25th Percentile: """ + str(round(df.loc[:,co2s[t]].quantile(q=0.25),1)) + """<br>
            Median: """ + str(round(df.loc[:,co2s[t]].quantile(q=0.50),1)) + """<br>
            75th Percentile: """ +str(round(df.loc[:,co2s[t]].quantile(q=0.75),1))  + """<br>
            95th Percentile: """ + str(round(df.loc[:,co2s[t]].quantile(q=0.95),1)) + """<br>
            Maximum: """ + str(round(df.loc[:,co2s[t]].max(),1)) + """<br></p>
            """
        graphs+="</div>"
        save(plo)
    if inrhs:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'inrhs')">Indoor Relative Humidity</button>"""
        graphs += """
        <div id="inrhs" class="tabcontent"><h3>Indoor Relative Humidity</h3><iframe src="inrh.html"></iframe>"""
        #add summary statistics
        #create bokeh plots
        output_file(folder+"/inrh.html")
        plo = figure(title="Indoor Relative Humidity", x_axis_label='Time', y_axis_label='%', x_axis_type='datetime')
        for t in range(0, len(inrhs)):
            plo.line(time, df[inrhs[t]].values, legend=inrhs[t], line_width=2, color =colarr[t])
            graphs+="""
            <p><b>"""+ inrhs[t] +"""<br></b>
            Mean: """+ str(round(df.loc[:,inrhs[t]].mean(),1))  + """<br>
            Minimum: """ + str(round(df.loc[:,inrhs[t]].min(),1)) + """<br>
            5th Percentile: """ + str(round(df.loc[:,inrhs[t]].quantile(q=0.05),1)) + """<br>
            25th Percentile: """ + str(round(df.loc[:,inrhs[t]].quantile(q=0.25),1)) + """<br>
            Median: """ + str(round(df.loc[:,inrhs[t]].quantile(q=0.50),1)) + """<br>
            75th Percentile: """ +str(round(df.loc[:,inrhs[t]].quantile(q=0.75),1))  + """<br>
            95th Percentile: """ + str(round(df.loc[:,inrhs[t]].quantile(q=0.95),1)) + """<br>
            Maximum: """ + str(round(df.loc[:,inrhs[t]].max(),1)) + """<br></p>
            """
        graphs+="</div>"
        save(plo)
    if outrhs:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'outrhs')">Outdoor Relative Humidity</button>"""
        graphs += """
        <div id="outrhs" class="tabcontent"><h3>Outdoor Relative Humidity</h3><iframe src="outrh.html"></iframe>"""
        #add summary statistics
        #create bokeh plots
        output_file(folder+"/outrh.html")
        plo = figure(title="Outdoor Relative Humidity", x_axis_label='Time', y_axis_label='%', x_axis_type='datetime')
        for t in range(0, len(outrhs)):
            plo.line(time, df[outrhs[t]].values, legend=outrhs[t], line_width=2, color =colarr[t])
            graphs+="""
            <p><b>"""+ outrhs[t] +"""<br></b>
            Mean: """+ str(round(df.loc[:,outrhs[t]].mean(),1))  + """<br>
            Minimum: """ + str(round(df.loc[:,outrhs[t]].min(),1)) + """<br>
            5th Percentile: """ + str(round(df.loc[:,outrhs[t]].quantile(q=0.05),1))  + """<br>
            25th Percentile: """ + str(round(df.loc[:,outrhs[t]].quantile(q=0.25),1))  + """<br>
            Median: """ + str(round(df.loc[:,outrhs[t]].quantile(q=0.50),1))  + """<br>
            75th Percentile: """ +str(round(df.loc[:,outrhs[t]].quantile(q=0.75),1))   + """<br>
            95th Percentile: """ + str(round(df.loc[:,outrhs[t]].quantile(q=0.95),1))  + """<br>
            Maximum: """ + str(round(df.loc[:,outrhs[t]].max(),1))  + """<br></p>
            """
        graphs+="</div>"
        save(plo)
    if forms:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'forms')">Formaldahyde</button>"""
        graphs += """
        <div id="forms" class="tabcontent"><h3>Formaldahyde</h3><iframe src="frm.html"></iframe>"""
        #add summary statistics
        #create bokeh plots
        output_file(folder+"/frm.html")
        plo = figure(title="Formaldahyde", x_axis_label='Time', y_axis_label='ppb', x_axis_type='datetime')
        for t in range(0, len(forms)):
            plo.line(time, df[forms[t]].values, legend=forms[t], line_width=2, color =colarr[t])
            graphs+="""
            <p><b>"""+ forms[t] +"""<br></b>
            Mean: """+ str(round(df.loc[:,forms[t]].mean(),1))  + """<br>
            Minimum: """ + str(round(df.loc[:,forms[t]].min(),1)) + """<br>
            5th Percentile: """ + str(round(df.loc[:,forms[t]].quantile(q=0.05),1)) + """<br>
            25th Percentile: """ + str(round(df.loc[:,forms[t]].quantile(q=0.25),1)) + """<br>
            Median: """ + str(round(df.loc[:,forms[t]].quantile(q=0.50),1)) + """<br>
            75th Percentile: """ +str(round(df.loc[:,forms[t]].quantile(q=0.75),1))  + """<br>
            95th Percentile: """ + str(round(df.loc[:,forms[t]].quantile(q=0.95),1)) + """<br>
            Maximum: """ + str(round(df.loc[:,forms[t]].max(),1)) + """<br></p>
            """
        graphs+="</div>"
        save(plo)
    if intemps:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'intemps')">Indoor Temperature</button>"""
        graphs += """
        <div id="intemps" class="tabcontent"><h3>Indoor Temperature</h3><iframe src="intemp.html"></iframe>"""
        #add summary statistics
        #create bokeh plots
        output_file(folder+"/intemp.html")
        plo = figure(title="Indoor Temperature", x_axis_label='Time', y_axis_label='oC', x_axis_type='datetime')
        for t in range(0, len(intemps)):
            plo.line(time, df[intemps[t]].values, legend=intemps[t], line_width=2, color =colarr[t])
            graphs+="""
            <p><b>"""+ intemps[t] +"""<br></b>
            Mean: """+ str(round(df.loc[:,intemps[t]].mean(),1))  + """<br>
            Minimum: """ + str(round(df.loc[:,intemps[t]].min(),1)) + """<br>
            5th Percentile: """ + str(round(df.loc[:,intemps[t]].quantile(q=0.05),1)) + """<br>
            25th Percentile: """ + str(round(df.loc[:,intemps[t]].quantile(q=0.25),1)) + """<br>
            Median: """ + str(round(df.loc[:,intemps[t]].quantile(q=0.50),1)) + """<br>
            75th Percentile: """ +str(round(df.loc[:,intemps[t]].quantile(q=0.75),1))  + """<br>
            95th Percentile: """ + str(round(df.loc[:,intemps[t]].quantile(q=0.95),1)) + """<br>
            Maximum: """ + str(round(df.loc[:,intemps[t]].max(),1)) + """<br></p>
            """
        graphs+="</div>"
        save(plo)
    if outtemps:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'outtemps')">Outdoor Temperature</button>"""
        graphs += """
        <div id="outtemps" class="tabcontent"><h3>Outdoor Temperature</h3><iframe src="outtemp.html"></iframe>"""
        #add summary statistics
        #create bokeh plots
        output_file(folder+"/outtemp.html")
        plo = figure(title="Outdoor Temperature", x_axis_label='Time', y_axis_label='oC', x_axis_type='datetime')
        for t in range(0, len(outtemps)):
            plo.line(time, df[outtemps[t]].values, legend=outtemps[t], line_width=2, color =colarr[t])
            graphs+="""
            <p><b>"""+ outtemps[t] +"""<br></b>
            Mean: """+ str(round(df.loc[:,outtemps[t]].mean(),1))  + """<br>
            Minimum: """ + str(round(df.loc[:,outtemps[t]].min(),1)) + """<br>
            5th Percentile: """ + str(round(df.loc[:,outtemps[t]].quantile(q=0.05),1)) + """<br>
            25th Percentile: """ + str(round(df.loc[:,outtemps[t]].quantile(q=0.25),1)) + """<br>
            Median: """ + str(round(df.loc[:,outtemps[t]].quantile(q=0.50),1)) + """<br>
            75th Percentile: """ +str(round(df.loc[:,outtemps[t]].quantile(q=0.75),1))  + """<br>
            95th Percentile: """ + str(round(df.loc[:,outtemps[t]].quantile(q=0.95),1)) + """<br>
            Maximum: """ + str(round(df.loc[:,outtemps[t]].max(),1)) + """<br></p>
            """
        graphs+="</div>"
        save(plo)
    if ibus:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'ibus')">iButton Temperature</button>"""
        graphs += """
        <div id="ibus" class="tabcontent"><h3>iButton Temperature</h3><iframe src="ibo.html"></iframe>"""
        #add summary statistics
        #create bokeh plots
        output_file(folder+"/ibo.html")
        plo = figure(title="iButton Temperature", x_axis_label='Time', y_axis_label='oC', x_axis_type='datetime')
        for t in range(0, len(ibus)):
            plo.line(time, df[ibus[t]].values, legend=ibus[t], line_width=2, color =colarr[t])
            graphs+="""
            <p><b>"""+ ibus[t] +"""<br></b>
            Mean: """+ str(round(df.loc[:,ibus[t]].mean(),1))  + """<br>
            Minimum: """ + str(round(df.loc[:,ibus[t]].min(),1)) + """<br>
            5th Percentile: """ + str(round(df.loc[:,ibus[t]].quantile(q=0.05),1)) + """<br>
            25th Percentile: """ + str(round(df.loc[:,ibus[t]].quantile(q=0.25),1)) + """<br>
            Median: """ + str(round(df.loc[:,ibus[t]].quantile(q=0.50),1)) + """<br>
            75th Percentile: """ +str(round(df.loc[:,ibus[t]].quantile(q=0.75),1))  + """<br>
            95th Percentile: """ + str(round(df.loc[:,ibus[t]].quantile(q=0.95),1)) + """<br>
            Maximum: """ + str(round(df.loc[:,ibus[t]].max(),1)) + """<br></p>
            """
        graphs+="</div>"
        save(plo)
    if mtrs:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'mtrs')">Motor On-Off</button>"""
        graphs += """
        <div id="mtrs" class="tabcontent"><h3>Motor On-Off</h3><iframe src="mtr.html"></iframe>"""
        #add summary statistics
        #create bokeh plots
        output_file(folder+"/mtr.html")
        plo = figure(title="Motor On-Off", x_axis_label='Time', y_axis_label='% time on', x_axis_type='datetime')
        for t in range(0, len(mtrs)):
            plo.line(time, df[mtrs[t]].values, legend=mtrs[t], line_width=2, color =colarr[t])
            graphs+="""
            <p><b>"""+ mtrs[t] +"""<br></b>
            Mean: """+ str(round(df.loc[:,mtrs[t]].mean(),1))  + """<br>
            Minimum: """ + str(round(df.loc[:,mtrs[t]].min(),1)) + """<br>
            5th Percentile: """ + str(round(df.loc[:,mtrs[t]].quantile(q=0.05),1)) + """<br>
            25th Percentile: """ + str(round(df.loc[:,mtrs[t]].quantile(q=0.25),1)) + """<br>
            Median: """ + str(round(df.loc[:,mtrs[t]].quantile(q=0.50),1)) + """<br>
            75th Percentile: """ +str(round(df.loc[:,mtrs[t]].quantile(q=0.75),1))  + """<br>
            95th Percentile: """ + str(round(df.loc[:,mtrs[t]].quantile(q=0.95),1)) + """<br>
            Maximum: """ + str(round(df.loc[:,mtrs[t]].max(),1)) + """<br></p>
            """
        graphs+="</div>"
        save(plo)
    if anms:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'anms')">Anemometer</button>"""
        graphs += """
        <div id="anms" class="tabcontent"><h3>Anemometer</h3><iframe src="anm.html"></iframe>"""
        #add summary statistics
        #create bokeh plots
        output_file(folder+"/anm.html")
        plo = figure(title="Anemometer", x_axis_label='Time', y_axis_label='m/s', x_axis_type='datetime')
        for t in range(0, len(anms)):
            plo.line(time, df[anms[t]].values, legend=anms[t], line_width=2, color =colarr[t])
            graphs+="""
            <p><b>"""+ anms[t] +"""<br></b>
            Mean: """+ str(round(df.loc[:,anms[t]].mean(),1))  + """<br>
            Minimum: """ + str(round(df.loc[:,anms[t]].min(),1)) + """<br>
            5th Percentile: """ + str(round(df.loc[:,anms[t]].quantile(q=0.05),1)) + """<br>
            25th Percentile: """ + str(round(df.loc[:,anms[t]].quantile(q=0.25),1)) + """<br>
            Median: """ + str(round(df.loc[:,anms[t]].quantile(q=0.50),1)) + """<br>
            75th Percentile: """ +str(round(df.loc[:,anms[t]].quantile(q=0.75),1))  + """<br>
            95th Percentile: """ + str(round(df.loc[:,anms[t]].quantile(q=0.95),1)) + """<br>
            Maximum: """ + str(round(df.loc[:,anms[t]].max(),1)) + """<br></p>
            """
        graphs+="</div>"
        save(plo)
    if doors:
        navlist+="""
        <button class="tablinks" onclick="openCity(event, 'doors')">Door State</button>"""
        graphs += """
        <div id="doors" class="tabcontent"><h3>Door State</h3><iframe src="sta.html"></iframe>"""
        #add summary statistics
        #create bokeh plots
        output_file(folder+"/sta.html")
        plo = figure(title="Door State", x_axis_label='Time', y_axis_label='Status', x_axis_type='datetime')
        for t in range(0, len(doors)):
            plo.line(time, df[doors[t]].values, legend=doors[t], line_width=2, color =colarr[t])
            graphs+="""
            <p><b>"""+ doors[t] +"""<br></b>
            Mean: """+ str(round(df.loc[:,doors[t]].mean(),1)) + """<br>
            Minimum: """ + str(round(df.loc[:,doors[t]].min(),1)) + """<br>
            5th Percentile: """ + str(round(df.loc[:,doors[t]].quantile(q=0.05),1)) + """<br>
            25th Percentile: """ + str(round(df.loc[:,doors[t]].quantile(q=0.25),1)) + """<br>
            Median: """ + str(round(df.loc[:,doors[t]].quantile(q=0.50),1)) + """<br>
            75th Percentile: """ +str(round(df.loc[:,doors[t]].quantile(q=0.75),1))  + """<br>
            95th Percentile: """ + str(round(df.loc[:,doors[t]].quantile(q=0.95),1)) + """<br>
            Maximum: """ + str(round(df.loc[:,doors[t]].max(),1)) + """<br></p>
            """
        graphs+="</div>"
        save(plo)

    tophtml = tophtml + navlist + "</div>" + graphs + """<script>
function openCity(evt, cityName) {
  var i, tabcontent, tablinks;
  tabcontent = document.getElementsByClassName("tabcontent");
  for (i = 0; i < tabcontent.length; i++) {
    tabcontent[i].style.display = "none";
  }
  tablinks = document.getElementsByClassName("tablinks");
  for (i = 0; i < tablinks.length; i++) {
    tablinks[i].className = tablinks[i].className.replace(" active", "");
  }
  document.getElementById(cityName).style.display = "block";
  evt.currentTarget.className += " active";
}
// Get the element with id="defaultOpen" and click on it
document.getElementById("defaultOpen").click();
</script>
</body>
</html> """
#     print(tophtml)
    hp = open(folder+'/index.html','w')
    hp.write(tophtml)
    hp.close()

debut 001
debut 003
debut 004
debut 005
debut 006
debut 007
debut 008
debut 009
debut 010
debut 011
debut 013
debut 015
debut 016
debut 017
debut 019
debut 021
debut 024
debut 025
debut 026
debut 027
debut 028
debut 029
debut 030
debut 031
debut 032
debut 034
debut 037
debut 038
debut 039
debut 040
debut 041
debut 042
debut 044
debut 045
debut 047
debut 048
debut 049
debut 050
debut 052
debut 053
debut 055
debut 058
debut 059
debut 063
debut 065
debut 066
debut 068
debut 101
debut 102
debut 104
debut 105
debut 106
debut 107
debut 108
debut 109
debut 110
debut 112
debut 113
debut 114
debut 115
debut 116
debut 117
debut 121
debut 122
debut 123
debut 124
debut 125
debut 127
debut 128
debut 129
debut 130
debut 201
debut 202
debut 203
debut 204
debut 205
debut 206
debut 207
debut 208
debut 301_OFF
debut 301_ON
debut 302_OFF
debut 302_ON
debut 303
debut 304_OFF
debut 304_ON
debut 305
debut 401
debut 402
debut 403
debut 404
debut 405
debut 406
debut 407_OFF
debut 407_ON
debut 408
debut 409
de